# Citations:
## The data used for this example: https://www.kaggle.com/datasets/fournierp/captcha-version-2-images
## Ensure that you place the .png files from this dataset into the samples folder

In [150]:
import numpy as np
import tensorflow as tf
import math
import os
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint


In [151]:
# We have images from Kaggles CAPTCHA images. Let's gather and resize them. Resizing images can help with computational complexity
images = os.listdir('./samples/')
image_data = []
for image in images:
    img = load_img('./samples/' + image, target_size=(64, 64))  # Resize images to desired dimensions
    img_array = img_to_array(img)
    image_data.append(img_array)
image_data = np.array(image_data)
#print(image_data)
image_data = image_data.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
print(np.shape(image_data[0]))


(64, 64, 3)


In [152]:
# We need to split our image data into training and test data, minimally. You can also split into training, test, and validation data if you choose
split_index = len(image_data) * .8
#print(split_index)
train = image_data[:int(split_index)]
test = image_data[int(split_index) + 1:]

In [ ]:
# We have to build our autoencoder. Our images have the shape (64, 64, 3). We want to save a potion of our images for validation. We will save 20% for this purpose
# We need to perform the following steps:
# 1. Set out input layer based on the shape of our data
# 2. Flatten our input layer to be fed into the Dense layers.
# 3. Encode the data to a lower dimension
# 4. Decode the layer back to the original dimension
# 5. Input our model parameters. This is our autoencoder
# 6. Compile the autoencoder
# 7. Train the autoencoder
# 8. Get predictions and the best model


input_layer = Input(shape=(64, 64, 3))
flattened_input = tf.keras.layers.Flatten()(input_layer)

encoded = Dense(32, activation='relu')(flattened_input)
decoded = Dense(64 * 64 * 3, activation='sigmoid')(encoded)

reshaped_output = tf.keras.layers.Reshape((64, 64, 3))(decoded)
autoencoder = Model(input_layer, reshaped_output)

model_checkpoint = ModelCheckpoint(
    filepath='./best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    verbose=0
)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(
    train, 
    train, 
    epochs=100, 
    batch_size=8,
    callbacks=[model_checkpoint],
    shuffle=True,
    validation_data=(test, test)
)

predictions = autoencoder.predict(
    test
)

display(test, predictions)

### Play with the model hyperparameters to see if you can improve accuracy